In [16]:
import pandas as pd
import sqlite3
import numpy as np

conn = sqlite3.connect(r"E:\Dev_Environment\equity-fundamental-analytics\data\gold\nse_equity_universe_gold.db")

stocks = pd.read_sql(
    "SELECT * FROM gold_equity_derivedmetrics",
    conn
)

rules = pd.read_excel(
    r"E:\Dev_Environment\equity-fundamental-analytics\sandbox\dim_metrics.xlsx",
    sheet_name="dim_relative"
)

MACRO = "Peak"
rules = rules[rules["macro_phase"] == MACRO]

In [17]:
stocks["relative_score"] = 0

In [18]:
def eval_vectorized(expr, df):
    return eval(
        expr,
        {"__builtins__": {}},
        df.to_dict("series")
    )

In [19]:
import operator

OPS = {
    ">": operator.gt,
    "<": operator.lt,
    "=": operator.eq,
    ">=": operator.ge,
    "<=": operator.le
}

for _, rule in rules.iterrows():

    sector = rule["sector_name"]
    metric = rule["metric_name"]
    threshold_expr = rule["threshold"]

    base_score = rule["score"]
    macro_weight = rule["macro_weight"]
    sector_weight = rule["sector_weight"]
    weighted_score = base_score * macro_weight * sector_weight
    op_fn = OPS[rule["operator"]]

    mask = stocks["sector"] == sector

    if mask.sum() == 0:
        continue

    try:
        right_values = eval_vectorized(
            threshold_expr,
            stocks.loc[mask]
        )

        passed = op_fn(
            stocks.loc[mask, metric],
            right_values
        )

        stocks.loc[mask, "relative_score"] += (
            passed.astype(int) * weighted_score
        )

    except Exception as e:
        # Rule failure = ignored, but engine continues
        continue

C:\Users\vishn\AppData\Local\Temp\ipykernel_10492\1663803108.py:39: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.    0.    0.    0.    0.    4.125 0.    0.    0.    4.125 0.    4.125
 0.    0.    4.125 0.    4.125 0.    0.    4.125 0.    4.125 0.    0.
 4.125 0.    0.    0.    0.    0.    4.125 0.    0.    0.    0.    0.
 0.    4.125 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 4.125 0.    0.    0.    0.    0.    0.    0.    4.125 4.125 0.    0.
 0.    0.    4.125 4.125 4.125 4.125 0.    0.    0.    0.    0.    0.
 0.    0.    0.    4.125 0.    0.    4.125 0.    0.    4.125 0.    0.
 0.    0.    4.125 0.    0.    4.125 4.125 0.    4.125 4.125 0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    4.125 4.125 0.    0.
 0.    0.    0.    0.    0.    0.    4.125 4.125 0.    0.    0.    4.125
 0.    4.125 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.

In [20]:
output = stocks[
    ["symbol", "sector", "relative_score"]
].copy()

output["macro_phase"] = MACRO

output.to_sql(
    "gold_scoring_engine",
    conn,
    if_exists="replace",
    index=False
)

conn.close()